In [1]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.python.data import AUTOTUNE
from torchmetrics.detection import mean_ap

from finalproject.data_loading import process_path
from finalproject.network import Net
from finalproject.loss import from_grid_to_coordinates, cxcywh_to_xyxy
import PIL.Image
from finalproject.utils import annotate_image
import torch

os.chdir("/home/user/src_wsl/mal2")


def eval(mean_ap, detection, label):
    det_boxes = torch.tensor(detection.numpy(), dtype=torch.float32)
    det_scores = torch.tensor([1.0], dtype=torch.float32)
    det_labels = torch.tensor([1], dtype=torch.int64)

    label_boxes = torch.tensor(label[0, :,0:4].numpy(), dtype=torch.float32)
    label_labels = torch.tensor([1], dtype=torch.int64)

    eval_detection = {
        "boxes": det_boxes,
        "scores": det_scores,
        "labels": det_labels
    }

    eval_label = {
        "boxes": label_boxes,
        "labels": label_labels
    }

    mean_ap.update([eval_detection], [eval_label])


with tf.device('/CPU:0'):
    grid_h = 1
    grid_w = 1
    batch_size = 1
    epochs = 20

    list_ds = tf.data.Dataset.list_files('finalproject/banana-detection/bananas_val/images/*.png', shuffle=False)
    val_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.batch(batch_size).prefetch(buffer_size=AUTOTUNE)
    ap = mean_ap.MeanAveragePrecision()
    model = Net()
    model.build((None, 224, 224, 3))
    model.load_weights('finalproject/saved_model/epoch_6.weights.h5')

    for i, (images, labels, file_path) in enumerate(val_ds):
        preds = model.predict(images)

        prediction = preds[0]
        prediction = from_grid_to_coordinates(prediction)
        if i % 5 == 0:
            img = PIL.Image.open(file_path[0].numpy().decode('utf-8'))
            img = annotate_image(img, targets=prediction[:, 0:4], scores=prediction[:, 4], labels=tf.constant([0]),
                                 normalized=True)
            plt.figure(figsize=(4, 4), facecolor='black')
            plt.imshow(img)
            plt.axis('off')
            plt.show()
        eval(ap, prediction[:, 0:4], cxcywh_to_xyxy(labels[0]))

    print(ap.compute())


I0000 00:00:1764240410.286426  123275 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13361 MB memory:  -> device: 0, name: AMD Radeon RX 9070 XT, pci bus id: 0000:03:00.0
2025-11-27 11:46:51.630149: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: Inner dimensions of output shape must match inner dimensions of updates shape. Output: [1,4] updates: [1,5]
2025-11-27 11:46:51.630218: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: Inner dimensions of output shape must match inner dimensions of updates shape. Output: [1,4] updates: [1,5]
2025-11-27 11:46:51.634312: W tensorflow/core/framework/op_kernel.cc:1829] UNKNOWN: InvalidArgumentError: {{function_node __wrapped__TensorScatterUpdate_device_/job:localhost/replica:0/task:0/device:GPU:0}} Inner dimensions of output shape must match inner dimensions of updates shape. Output:

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} InvalidArgumentError: {{function_node __wrapped____EagerConst_device_/job:localhost/replica:0/task:0/device:GPU:0}} Inner dimensions of output shape must match inner dimensions of updates shape. Output: [1,4] updates: [1,5]
Traceback (most recent call last):

  File "/home/user/.local/lib/python3.12/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/user/.local/lib/python3.12/site-packages/tensorflow/python/ops/script_ops.py", line 145, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/user/.local/lib/python3.12/site-packages/tensorflow/python/ops/script_ops.py", line 152, in _call
    ret = self._func(*args)
          ^^^^^^^^^^^^^^^^^

  File "/home/user/.local/lib/python3.12/site-packages/tensor

In [ ]:
from finalproject.loss import from_grid_to_coordinates
import PIL.Image
from finalproject.utils import annotate_image
from matplotlib import pyplot as plt


prediction = preds[0]
prediction = from_grid_to_coordinates(prediction)
img = PIL.Image.open(file_path[0].numpy().decode('utf-8'))
img = annotate_image(img, targets=prediction[:, 0:4], scores=prediction[:, 4], labels=tf.constant([0]), normalized=True)
plt.figure(figsize=(4, 4), facecolor='black')
plt.imshow(img)
plt.axis('off')
plt.show()